In [4]:

# premium_api_enabled = True/False
premium_api_enabled = False
# premium_type = '30day'/'fullarchive'
premium_type = '30day'

In [2]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pymongo
import base64
import requests
import urllib.parse
from IPython.core.debugger import set_trace

In [2]:
base_url = 'https://api.twitter.com/'
auth_url = '{}oauth2/token'.format(base_url)
client_key = 'h1FMGukUb2opdWyp6e5h4wXsJ'
client_secret = 'Bd3604rdoBGdsVdBL6fubOpyYL44L0uJNpBp5D63IOe5k4h6Rg'

In [5]:
# https://api.twitter.com/1.1/tweets/search/fullarchive/:label.json

if premium_api_enabled:
    search_url = '{}1.1/tweets/search/{}/test01.json'.format(base_url, premium_type)
else:
    search_url = '{}1.1/search/tweets.json'.format(base_url)
print(search_url)

https://api.twitter.com/1.1/search/tweets.json


In [6]:
key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

auth_headers = {
    'Authorization': 'Basic {}'.format(b64_encoded_key),
    'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'
}
auth_data = {
    'grant_type': 'client_credentials'
}
auth_resp = requests.post(auth_url, headers=auth_headers, data=auth_data)
access_token = auth_resp.json()['access_token']

auth_headers = {
    'Authorization': 'Bearer {}'.format(access_token)    
}

In [7]:
dbUrl = "mongodb://localhost:27017/"
dbName = "test"
collectionName = "testing"

In [8]:
dbClient = pymongo.MongoClient(dbUrl)
db = dbClient[dbName]
collection = db[collectionName]

In [9]:
if premium_api_enabled:
    # premium API search_params
    search_params = {
        'query': 'covid19',
    #     'tag': '',
    #     'fromDate': 201512220000
    #     'toDate': 201712220000
    }
else:
    # standard API search_params
    search_params = {
        'q': 'uppcl',
        'result_type': 'recent', # mixed/recent/popular
        'locale': 'en',
        'count': 100
    }
print(search_params)

{'q': 'uppcl', 'result_type': 'recent', 'locale': 'en', 'count': 100}


In [10]:
def fetchTweets():
    search_resp = requests.get(search_url, headers=auth_headers, params=search_params)
    return search_resp

In [11]:
def insertTweetsIntoDb(search_resp):
    count = 0
    if premium_api_enabled:
        tweet_data = search_resp.json()['results']
    else:
        tweet_data = search_resp.json()['statuses']
    for x in tweet_data:
        try:
            x['_id'] = x['id']
            collection.insert_one(x)
            count = count + 1
        except pymongo.errors.DuplicateKeyError:
            # skip tweet because it already exists in new collection
            continue
    return count

In [12]:

tweets_inserted = 0
for i in range(1):
    search_resp = fetchTweets()
    tweets_inserted = tweets_inserted + insertTweetsIntoDb(search_resp)
print('{} tweets inserted.'.format(tweets_inserted))

100 tweets inserted.
